# This notebook generates a CSV of points with large prediction errors in V15 or V20

In [2]:
import pandas as pd
df = pd.read_pickle('tmp_data/clean_yearly_data.pkl')

In [17]:
df

,latitude_ERA,longitude_ERA,MODIS_LST,V15_prediction,V20_prediction,V20X_prediction,V15X_prediction,V15_error,V20_error,V20X_error,...,V15_dl,V20_dl,percentage_change_in_dl,cl_change_is_significant,si10_change_is_significant,cvh_change_is_significant,cvl_change_is_significant,lsm_change_is_significant,dl_change_is_significant,bitstring
0,-70.117048,-75.75,256.971771,260.839691,258.764496,258.308044,260.245789,5.176608,4.774697,4.901756,...,68.036133,222.733398,2.273751,False,True,False,False,True,True,000111
1,-70.117048,-75.00,256.868408,260.022491,257.034821,254.135437,262.143463,4.589613,4.524141,5.674824,...,25.000000,105.971680,3.238867,False,True,False,False,False,True,000101
2,-70.117048,-72.00,255.929306,258.782562,257.770081,256.421112,257.124664,5.802632,5.922895,5.830910,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
3,-70.117048,-71.25,254.781509,258.479736,257.310944,257.045654,257.465698,5.732193,5.754484,5.606140,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
4,-70.117048,-70.50,255.270538,258.035889,258.086243,256.977478,259.335144,6.000848,5.888017,6.004581,...,25.000000,10.000000,-0.600000,False,True,False,False,False,True,000101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149805,70.117048,168.00,264.279388,264.181641,264.485321,264.104553,261.404938,3.836673,3.708954,3.456014,...,30.217773,31.002930,0.025983,False,False,False,False,False,False,000000
149806,70.117048,168.75,263.243958,263.457367,263.287903,263.835266,260.608521,3.779803,3.540926,3.669104,...,19.599609,18.327148,-0.064923,False,False,False,False,False,False,000000
149807,70.117048,170.25,266.316162,262.494171,262.690735,262.242371,264.249634,5.866361,5.818576,6.012778,...,20.901367,20.796875,-0.004999,False,False,False,False,False,False,000000
149808,70.117048,171.00,266.383087,264.283325,264.261658,264.064941,263.495117,5.195374,5.332411,5.318110,...,28.710938,27.846680,-0.030102,False,False,False,False,False,False,000000


In [27]:
#Get the points where the V15 error is large
V15error = df.query('V15_error > 5.1')


#Get the points where the V20 error is large
V20error = df.query('V20_error > 5.1')



,latitude_ERA,longitude_ERA,MODIS_LST,V15_prediction,V20_prediction,V20X_prediction,V15X_prediction,V15_error,V20_error,V20X_error,...,V15_dl,V20_dl,percentage_change_in_dl,cl_change_is_significant,si10_change_is_significant,cvh_change_is_significant,cvl_change_is_significant,lsm_change_is_significant,dl_change_is_significant,bitstring
0,-70.117048,-75.75,256.971771,260.839691,258.764496,258.308044,260.245789,5.176608,4.774697,4.901756,...,68.036133,222.733398,2.273751,False,True,False,False,True,True,000111
2,-70.117048,-72.00,255.929306,258.782562,257.770081,256.421112,257.124664,5.802632,5.922895,5.830910,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
3,-70.117048,-71.25,254.781509,258.479736,257.310944,257.045654,257.465698,5.732193,5.754484,5.606140,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
4,-70.117048,-70.50,255.270538,258.035889,258.086243,256.977478,259.335144,6.000848,5.888017,6.004581,...,25.000000,10.000000,-0.600000,False,True,False,False,False,True,000101
5,-70.117048,-69.75,254.160919,254.956985,255.068756,253.595154,255.634949,6.404216,6.179307,6.414968,...,25.000000,10.000000,-0.600000,False,True,False,False,False,True,000101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149659,70.117048,-24.75,256.992676,259.825653,259.268250,259.919342,260.262024,5.168821,4.797196,4.672724,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
149660,70.117048,-24.00,258.501343,265.115753,265.274750,261.810822,257.686798,7.608796,7.365232,4.990395,...,403.301758,123.036133,-0.694928,True,False,False,False,True,True,100110
149663,70.117048,18.75,273.048859,274.975769,275.161072,275.559296,265.583679,6.089649,5.667446,5.590523,...,86.179688,34.093750,-0.604388,False,False,False,False,False,True,000100
149807,70.117048,170.25,266.316162,262.494171,262.690735,262.242371,264.249634,5.866361,5.818576,6.012778,...,20.901367,20.796875,-0.004999,False,False,False,False,False,False,000000


In [28]:
df.query('V20_error > 5.1')

,latitude_ERA,longitude_ERA,MODIS_LST,V15_prediction,V20_prediction,V20X_prediction,V15X_prediction,V15_error,V20_error,V20X_error,...,V15_dl,V20_dl,percentage_change_in_dl,cl_change_is_significant,si10_change_is_significant,cvh_change_is_significant,cvl_change_is_significant,lsm_change_is_significant,dl_change_is_significant,bitstring
2,-70.117048,-72.00,255.929306,258.782562,257.770081,256.421112,257.124664,5.802632,5.922895,5.830910,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
3,-70.117048,-71.25,254.781509,258.479736,257.310944,257.045654,257.465698,5.732193,5.754484,5.606140,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
4,-70.117048,-70.50,255.270538,258.035889,258.086243,256.977478,259.335144,6.000848,5.888017,6.004581,...,25.000000,10.000000,-0.600000,False,True,False,False,False,True,000101
5,-70.117048,-69.75,254.160919,254.956985,255.068756,253.595154,255.634949,6.404216,6.179307,6.414968,...,25.000000,10.000000,-0.600000,False,True,False,False,False,True,000101
6,-70.117048,-69.00,255.507721,253.973679,254.695847,253.855438,257.955963,5.693578,5.600639,5.959773,...,25.000000,112.903320,3.516133,False,True,False,False,False,True,000101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149660,70.117048,-24.00,258.501343,265.115753,265.274750,261.810822,257.686798,7.608796,7.365232,4.990395,...,403.301758,123.036133,-0.694928,True,False,False,False,True,True,100110
149661,70.117048,-23.25,262.689972,265.529877,269.570770,263.857452,260.656738,4.999030,7.248564,3.531274,...,25.000000,270.564453,9.822578,False,True,False,False,False,True,000101
149663,70.117048,18.75,273.048859,274.975769,275.161072,275.559296,265.583679,6.089649,5.667446,5.590523,...,86.179688,34.093750,-0.604388,False,False,False,False,False,True,000100
149807,70.117048,170.25,266.316162,262.494171,262.690735,262.242371,264.249634,5.866361,5.818576,6.012778,...,20.901367,20.796875,-0.004999,False,False,False,False,False,False,000000


In [15]:
df_nocoean = df.query("V15_lsm == 0.0")

In [16]:
df_nocoean

,latitude_ERA,longitude_ERA,MODIS_LST,V15_prediction,V20_prediction,V20X_prediction,V15X_prediction,V15_error,V20_error,V20X_error,...,V15_dl,V20_dl,percentage_change_in_dl,cl_change_is_significant,si10_change_is_significant,cvh_change_is_significant,cvl_change_is_significant,lsm_change_is_significant,dl_change_is_significant,bitstring
1338,-68.149839,153.360000,256.032227,256.399780,255.423935,256.035126,255.401978,3.775687,3.709305,3.270363,...,420.959961,418.859375,-0.004990,False,False,False,False,False,False,000000
1948,-66.463660,-67.333333,262.617218,264.329559,263.986053,263.515411,263.394714,4.489912,4.586503,4.711741,...,302.000000,308.242188,0.020669,False,False,False,False,False,False,000000
2023,-66.182630,-67.333333,266.413239,264.887817,264.758545,264.576599,264.562683,3.760714,3.807927,4.443919,...,344.416992,326.985352,-0.050612,False,False,False,False,False,False,000000
2187,-63.372331,-62.400000,263.676178,266.297852,266.066132,265.628662,264.267212,4.577663,4.621567,4.421271,...,158.087891,134.757812,-0.147577,False,False,False,False,False,True,000100
2188,-63.372331,-61.800000,264.405548,266.706573,265.722565,265.416595,264.925110,3.942123,3.993344,4.067698,...,689.269531,637.153320,-0.075611,False,False,False,False,False,False,000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143202,65.620570,-121.875000,265.696747,268.124695,267.509644,267.597534,267.332611,3.176112,2.994736,3.094981,...,74.180664,99.114258,0.336120,False,False,False,False,False,True,000100
143679,65.901600,-120.666667,264.995209,266.926086,266.494385,266.544067,265.611023,2.989954,3.200369,3.184757,...,95.405273,100.000000,0.048160,False,False,False,False,False,False,000000
144127,66.182630,-120.666667,264.998688,266.616730,266.156250,266.227600,266.066315,3.014317,3.043509,3.078618,...,97.800781,100.000000,0.022487,False,False,False,False,False,False,000000
144579,66.463660,-121.333333,264.887756,266.239044,265.714783,265.815521,265.262085,2.880560,2.965111,2.965191,...,85.721680,89.360352,0.042448,False,False,False,False,False,False,000000


In [ ]:
& V15_cl == 0.0 & V20_cl == 0.0 & V20_lsm == 0.0